In [7]:
from datetime import datetime
from types import NoneType
from send2trash import send2trash
from tqdm import tqdm
from model import all_dirs, ext, all_types, Base, Media, session, engine
from pathlib import Path
from pandas import DataFrame, read_sql
from itertools import chain
from exiftool import ExifToolHelper
from cv2 import VideoCapture, CAP_PROP_FRAME_COUNT, CAP_PROP_POS_FRAMES, COLOR_BGR2RGB, cvtColor
from PIL import Image
from xxhash import xxh64
from pillow_heif import register_heif_opener
from humanize import naturalsize

register_heif_opener()

In [8]:
df = read_sql("google_photos", con=engine)
df["path"] = df["path"].apply(Path) #type: ignore
df

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data
0,IMG_20190501_115040.jpg,D:\RAW\Photos - Backup\2019\05\IMG_20190501_11...,jpg,1070346,1.1 MB,image,2019-05-01 11:50:40,2019-05-01 11:50:40,4cd351143a62a1c0,2b6f99a8f9a8b39f,"{'EXIF:ISO': 500, 'EXIF:Make': 'HUAWEI', 'EXIF..."
1,IMG_20211218_175925.jpg,D:\RAW\Photos - Backup\2021\12\IMG_20211218_17...,jpg,4065131,4.1 MB,image,2021-12-18 17:59:25,2021-12-18 17:59:25,e5c2d2523fe318ea,adc9f78bab8c7d20,"{'EXIF:ISO': 126, 'JSON:Hdr': 'off', 'EXIF:Mak..."
2,Screenshot_2021-12-09-13-58-27-75_6012fa4d4dde...,D:\RAW\Photos - Backup\2021\12\Screenshot_2021...,jpg,85133,85.1 kB,image,2021-12-09 13:58:27,2021-12-09 13:58:27,8d9940fc4b550122,b1eaef385a445252,{'SourceFile': 'D:/RAW/Photos - Backup/2021/12...
3,Harshad Salunke 20181010_002855.jpg,D:\RAW\Photos - Backup\2018\10\Harshad Salunke...,jpg,43384,43.4 kB,image,2018-10-10 00:28:55,2018-10-10 00:28:55,bc18497f9f6fad20,9d4a1acc618905b1,{'SourceFile': 'D:/RAW/Photos - Backup/2018/10...
4,B612_20180329_114944.jpg,D:\RAW\Photos\2018\03\B612_20180329_114944.jpg,jpg,168812,168.8 kB,image,2018-03-29 11:49:44,2018-03-29 11:49:44,c2a7f9f3f85ce9b3,5af175676bb7ddea,{'SourceFile': 'D:/RAW/Photos/2018/03/B612_201...
...,...,...,...,...,...,...,...,...,...,...,...
66335,preview.jpg,D:\RAW\I2201\Pictures\Telegram\preview.jpg,jpg,351966,352.0 kB,image,2024-12-19 03:06:49,2024-07-28 01:10:10,cbd25cdd09f68700,f321840db7c11ca1,{'SourceFile': 'D:/RAW/I2201/Pictures/Telegram...
66336,Screenrecording_20221216_204737~2.mp4,D:\RAW\I2201\Pictures\Templates\Screenrecordin...,mp4,1489636,1.5 MB,video,2024-12-19 03:06:51,2022-12-16 15:17:46,4732478988933e64,eb28559b40bac270,{'SourceFile': 'D:/RAW/I2201/Pictures/Template...
66337,Screenrecording_20230402_232520~2.mp4,D:\RAW\I2201\Pictures\Templates\Screenrecordin...,mp4,15777838,15.8 MB,video,2024-12-19 03:06:54,2023-04-02 17:56:11,21375a2d6196cea4,1135fb0c280e6878,{'SourceFile': 'D:/RAW/I2201/Pictures/Template...
66338,IMG_20241219_215007_147.jpg,D:\RAW\I2201\Pictures\Threads\IMG_20241219_215...,jpg,49099,49.1 kB,image,2024-12-23 02:22:11,NaT,1ae1a5051be6f4e5,edd30e81a20f4e1b,{'SourceFile': 'D:/RAW/I2201/Pictures/Threads/...


In [9]:
# exif tags

df["exif_count"] = df["exif_data"].apply(len)
df

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data,exif_count
0,IMG_20190501_115040.jpg,D:\RAW\Photos - Backup\2019\05\IMG_20190501_11...,jpg,1070346,1.1 MB,image,2019-05-01 11:50:40,2019-05-01 11:50:40,4cd351143a62a1c0,2b6f99a8f9a8b39f,"{'EXIF:ISO': 500, 'EXIF:Make': 'HUAWEI', 'EXIF...",106
1,IMG_20211218_175925.jpg,D:\RAW\Photos - Backup\2021\12\IMG_20211218_17...,jpg,4065131,4.1 MB,image,2021-12-18 17:59:25,2021-12-18 17:59:25,e5c2d2523fe318ea,adc9f78bab8c7d20,"{'EXIF:ISO': 126, 'JSON:Hdr': 'off', 'EXIF:Mak...",103
2,Screenshot_2021-12-09-13-58-27-75_6012fa4d4dde...,D:\RAW\Photos - Backup\2021\12\Screenshot_2021...,jpg,85133,85.1 kB,image,2021-12-09 13:58:27,2021-12-09 13:58:27,8d9940fc4b550122,b1eaef385a445252,{'SourceFile': 'D:/RAW/Photos - Backup/2021/12...,57
3,Harshad Salunke 20181010_002855.jpg,D:\RAW\Photos - Backup\2018\10\Harshad Salunke...,jpg,43384,43.4 kB,image,2018-10-10 00:28:55,2018-10-10 00:28:55,bc18497f9f6fad20,9d4a1acc618905b1,{'SourceFile': 'D:/RAW/Photos - Backup/2018/10...,74
4,B612_20180329_114944.jpg,D:\RAW\Photos\2018\03\B612_20180329_114944.jpg,jpg,168812,168.8 kB,image,2018-03-29 11:49:44,2018-03-29 11:49:44,c2a7f9f3f85ce9b3,5af175676bb7ddea,{'SourceFile': 'D:/RAW/Photos/2018/03/B612_201...,54
...,...,...,...,...,...,...,...,...,...,...,...,...
66335,preview.jpg,D:\RAW\I2201\Pictures\Telegram\preview.jpg,jpg,351966,352.0 kB,image,2024-12-19 03:06:49,2024-07-28 01:10:10,cbd25cdd09f68700,f321840db7c11ca1,{'SourceFile': 'D:/RAW/I2201/Pictures/Telegram...,61
66336,Screenrecording_20221216_204737~2.mp4,D:\RAW\I2201\Pictures\Templates\Screenrecordin...,mp4,1489636,1.5 MB,video,2024-12-19 03:06:51,2022-12-16 15:17:46,4732478988933e64,eb28559b40bac270,{'SourceFile': 'D:/RAW/I2201/Pictures/Template...,72
66337,Screenrecording_20230402_232520~2.mp4,D:\RAW\I2201\Pictures\Templates\Screenrecordin...,mp4,15777838,15.8 MB,video,2024-12-19 03:06:54,2023-04-02 17:56:11,21375a2d6196cea4,1135fb0c280e6878,{'SourceFile': 'D:/RAW/I2201/Pictures/Template...,72
66338,IMG_20241219_215007_147.jpg,D:\RAW\I2201\Pictures\Threads\IMG_20241219_215...,jpg,49099,49.1 kB,image,2024-12-23 02:22:11,NaT,1ae1a5051be6f4e5,edd30e81a20f4e1b,{'SourceFile': 'D:/RAW/I2201/Pictures/Threads/...,49


File Hash Duplicates

In [10]:
dup = df[df.duplicated(subset=["file_hash"], keep=False)].copy().reset_index(drop=True).sort_values(by=["file_hash", "exif_date", "exif_count", "modify_date"], ascending=[True, True, False, True])
dup

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data,exif_count


In [11]:
delete = dup[dup.duplicated(subset=["file_hash"])]
delete

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data,exif_count


In [12]:
keep = dup[~dup.duplicated(subset=["file_hash"])]
keep

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data,exif_count


In [13]:
# delete["path"].apply(send2trash)
a = list(map(send2trash, tqdm(delete["path"], total=len(delete))))

0it [00:00, ?it/s]


In [14]:
# ! python refresh_db.py

Media Hash Duplicates

In [15]:
dup = df[df.duplicated(subset=["media_hash"], keep=False)].copy().reset_index(drop=True).sort_values(by=["media_hash", "exif_date", "exif_count", "modify_date"], ascending=[True, True, False, True])
dup

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data,exif_count
1818,pink.yyy__-04022022-0001.jpg,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,jpg,137764,137.8 kB,image,2025-03-05 20:38:58.024,2022-02-04 07:00:10,f5f877e25bd09fb7,00188d6f58d12851,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,38
1359,pink.yyy__-24-03-2023-0089.webp,D:\RAW\I2201\Pictures\Instander\pink.yyy__-24-...,webp,137514,137.5 kB,image,2024-12-19 02:55:58.000,NaT,939f1ce785cf7439,00188d6f58d12851,{'SourceFile': 'D:/RAW/I2201/Pictures/Instande...,26
1966,hima__honey-28-05-2023-0001.jpg,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,jpg,123246,123.2 kB,image,2025-03-05 20:40:30.621,2023-05-28 12:36:40,02ab554cffec282e,00462aeba46c1c16,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,38
1106,hima__honey-28-05-2023-0001.jpg,D:\RAW\I2201\Pictures\Instander\hima__honey-28...,jpg,122996,123.0 kB,image,2024-12-19 02:55:09.000,NaT,cdcaea6fbfa8355d,00462aeba46c1c16,{'SourceFile': 'D:/RAW/I2201/Pictures/Instande...,26
2696,reddi_nandini-21012024-0002.mp4,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,mp4,7364715,7.4 MB,video,2025-03-05 21:34:29.243,2024-01-21 17:40:00,2432c1b24b9c5f56,0078693ce33b587d,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,73
...,...,...,...,...,...,...,...,...,...,...,...,...
873,shraddha_29_-20221217-0001.jpg,D:\RAW\I2201\Instagram\shraddha_29_-20221217-0...,jpg,388184,388.2 kB,image,2024-12-19 02:53:10.000,NaT,9b09190cc0ab95bb,ffd380ab177f3041,{'SourceFile': 'D:/RAW/I2201/Instagram/shraddh...,26
2682,nischitha_jaggi-23012024-0001.mp4,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,mp4,622124,622.1 kB,video,2025-03-05 21:30:00.372,2024-01-23 13:49:25,7dfdad2efdaf3f04,ffd4073bdbee265a,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,73
1269,nischitha_jaggi-23012024-0001.mp4,D:\RAW\I2201\Pictures\Instander\nischitha_jagg...,mp4,619036,619.0 kB,video,2024-12-19 02:55:44.000,NaT,fad087221c4e408a,ffd4073bdbee265a,{'SourceFile': 'D:/RAW/I2201/Pictures/Instande...,68
2606,yadniiip-20231222-0011.jpg,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,jpg,172985,173.0 kB,image,2025-03-05 18:31:52.450,2023-12-22 23:21:37,a2a78217e5bc0f16,fffe06ea0c0f98cd,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,38


In [16]:
delete = dup[dup.duplicated(subset=["media_hash"])]
delete

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data,exif_count
1359,pink.yyy__-24-03-2023-0089.webp,D:\RAW\I2201\Pictures\Instander\pink.yyy__-24-...,webp,137514,137.5 kB,image,2024-12-19 02:55:58,NaT,939f1ce785cf7439,00188d6f58d12851,{'SourceFile': 'D:/RAW/I2201/Pictures/Instande...,26
1106,hima__honey-28-05-2023-0001.jpg,D:\RAW\I2201\Pictures\Instander\hima__honey-28...,jpg,122996,123.0 kB,image,2024-12-19 02:55:09,NaT,cdcaea6fbfa8355d,00462aeba46c1c16,{'SourceFile': 'D:/RAW/I2201/Pictures/Instande...,26
1394,reddi_nandini-21012024-0002.mp4,D:\RAW\I2201\Pictures\Instander\reddi_nandini-...,mp4,7361627,7.4 MB,video,2024-12-19 02:56:07,NaT,fdd3e9dafb25e8a1,0078693ce33b587d,{'SourceFile': 'D:/RAW/I2201/Pictures/Instande...,68
793,retroserie-20230715-0003.jpg,D:\RAW\I2201\Instagram\retroserie-20230715-000...,jpg,63504,63.5 kB,image,2024-12-19 02:52:57,NaT,cce205cdba94a403,00871029d3e25b57,{'SourceFile': 'D:/RAW/I2201/Instagram/retrose...,26
922,yadniiip-20230107-0002.jpg,D:\RAW\I2201\Instagram\yadniiip-20230107-0002.jpg,jpg,256417,256.4 kB,image,2024-12-19 02:53:27,NaT,3bd993d427efcf24,00b0095384daabe5,{'SourceFile': 'D:/RAW/I2201/Instagram/yadniii...,26
...,...,...,...,...,...,...,...,...,...,...,...,...
1134,kalpanashrestha170-20240306-0001.mp4,D:\RAW\I2201\Pictures\Instander\kalpanashresth...,mp4,6084734,6.1 MB,video,2024-12-19 02:55:15,NaT,2142a1cfe8f4c9c5,ff6cde8e52dcbc58,{'SourceFile': 'D:/RAW/I2201/Pictures/Instande...,68
575,chill.bro_3-20221215-0001~2.mp4,D:\RAW\I2201\Instagram\chill.bro_3-20221215-00...,mp4,267623,267.6 kB,video,2024-12-19 02:52:22,NaT,a5fec57653eb5a94,ff89a359f9696fbf,{'SourceFile': 'D:/RAW/I2201/Instagram/chill.b...,73
873,shraddha_29_-20221217-0001.jpg,D:\RAW\I2201\Instagram\shraddha_29_-20221217-0...,jpg,388184,388.2 kB,image,2024-12-19 02:53:10,NaT,9b09190cc0ab95bb,ffd380ab177f3041,{'SourceFile': 'D:/RAW/I2201/Instagram/shraddh...,26
1269,nischitha_jaggi-23012024-0001.mp4,D:\RAW\I2201\Pictures\Instander\nischitha_jagg...,mp4,619036,619.0 kB,video,2024-12-19 02:55:44,NaT,fad087221c4e408a,ffd4073bdbee265a,{'SourceFile': 'D:/RAW/I2201/Pictures/Instande...,68


In [17]:
keep = dup[~dup.duplicated(subset=["media_hash"])]
keep

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data,exif_count
1818,pink.yyy__-04022022-0001.jpg,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,jpg,137764,137.8 kB,image,2025-03-05 20:38:58.024,2022-02-04 07:00:10,f5f877e25bd09fb7,00188d6f58d12851,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,38
1966,hima__honey-28-05-2023-0001.jpg,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,jpg,123246,123.2 kB,image,2025-03-05 20:40:30.621,2023-05-28 12:36:40,02ab554cffec282e,00462aeba46c1c16,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,38
2696,reddi_nandini-21012024-0002.mp4,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,mp4,7364715,7.4 MB,video,2025-03-05 21:34:29.243,2024-01-21 17:40:00,2432c1b24b9c5f56,0078693ce33b587d,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,73
68,retroserie-20230715-0003.jpg,C:\Users\seshu\Pictures\Share\3rd\retroserie-2...,jpg,63754,63.8 kB,image,2023-07-15 02:24:57.000,2023-07-15 02:24:57,cc45e11e87112535,00871029d3e25b57,{'SourceFile': 'C:/Users/seshu/Pictures/Share/...,38
2591,yadniiip-20230107-0002.jpg,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,jpg,256667,256.7 kB,image,2025-03-05 20:42:46.154,2023-01-07 17:56:34,fde54f72606068e2,00b0095384daabe5,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,38
...,...,...,...,...,...,...,...,...,...,...,...,...
2653,kalpanashrestha170-20240306-0001.mp4,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,mp4,6087822,6.1 MB,video,2025-03-05 18:31:04.828,2024-03-06 17:10:25,18b62d7ee37626d7,ff6cde8e52dcbc58,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,73
32,chill.bro_3-20221215-0001~2.mp4,C:\Users\seshu\Pictures\Share\3rd\chill.bro_3-...,mp4,270711,270.7 kB,video,2022-12-15 09:20:52.000,2022-12-15 03:50:52,5222ebc8f1bac0e5,ff89a359f9696fbf,{'SourceFile': 'C:/Users/seshu/Pictures/Share/...,78
1841,shraddha_29_-20221217-0001.jpg,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,jpg,388434,388.4 kB,image,2025-03-05 18:30:22.031,2022-12-17 19:15:52,6178b3d6af1236d8,ffd380ab177f3041,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,38
2682,nischitha_jaggi-23012024-0001.mp4,D:\RAW\seshu80743\Takeout\Google Photos\Photos...,mp4,622124,622.1 kB,video,2025-03-05 21:30:00.372,2024-01-23 13:49:25,7dfdad2efdaf3f04,ffd4073bdbee265a,{'SourceFile': 'D:/RAW/seshu80743/Takeout/Goog...,73


In [18]:
# delete["path"].apply(send2trash)
a = list(map(send2trash, tqdm(delete["path"], total=len(delete))))

100%|██████████| 1399/1399 [00:09<00:00, 145.28it/s]
